In [25]:
import json
import os
import sys
import pickle
import re

import numpy as np
import pandas as pd
from networks import DualNet, DualNetEndToEnd, PrimalNet, PrimalNetEndToEnd, PrimalNetEndToEnd2
import torch

def evaluate(data, primal_net, test_indices):        
    X = data.X[test_indices]
    Y_target = data.opt_targets["y_operational"][test_indices]
    
    # Forward pass through networks
    Y = primal_net(X)

    ineq_dist = data.ineq_dist(X, Y)
    eq_resid = data.eq_resid(X, Y)

    relative_ineq_dist = data.relative_ineq_dist(X, Y)
    relative_eq_resid = data.relative_eq_resid(X, Y)

    # Convert lists to arrays for easier handling
    obj_values = data.obj_fn(X, Y).detach().numpy()
    ineq_max_vals = torch.max(ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    ineq_mean_vals = torch.mean(ineq_dist, dim=1).detach().numpy()
    eq_max_vals = torch.max(torch.abs(eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    eq_mean_vals = torch.mean(torch.abs(eq_resid), dim=1).detach().numpy()

    relative_ineq_max_vals = torch.max(relative_ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    relative_ineq_mean_vals = torch.mean(relative_ineq_dist, dim=1).detach().numpy()
    relative_eq_max_vals = torch.max(torch.abs(relative_eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    relative_eq_mean_vals = torch.mean(torch.abs(relative_eq_resid), dim=1).detach().numpy()

    known_obj = data.obj_fn(X, Y_target).detach().numpy()
    # obj_values is negative
    opt_gap = (obj_values - known_obj)/np.abs(known_obj) * 100

    return np.mean(obj_values), np.mean(known_obj), np.mean(opt_gap), np.mean(ineq_max_vals), np.mean(relative_ineq_max_vals), np.mean(ineq_mean_vals), np.mean(relative_ineq_mean_vals), np.mean(eq_max_vals), np.mean(relative_eq_max_vals), np.mean(eq_mean_vals), np.mean(relative_eq_mean_vals)

def dual_evaluate_QP(data, dual_net):
    X = data.X[data.test_indices]
    target_mu = data.mu[data.test_indices]
    target_lamb = data.lamb[data.test_indices]
    # Forward pass through networks
    mu, lamb = dual_net(X)

    obj_values = data.dual_obj_fn(X, mu, lamb).detach().numpy()
    known_obj = data.dual_obj_fn(X, target_mu, target_lamb).detach().numpy()
    dual_ineq_dist = data.dual_ineq_dist(mu, lamb)
    dual_eq_resid = data.dual_eq_resid(mu, lamb)

    opt_gap = (obj_values - known_obj)/np.abs(known_obj) * 100

    ineq_max_vals = torch.max(dual_ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    # eq_max_vals = torch.max(torch.abs(dual_eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    ineq_mean_vals = torch.mean(dual_ineq_dist, dim=1).detach().numpy()
    # eq_mean_vals = torch.mean(torch.abs(dual_eq_resid), dim=1).detach().numpy()

    return np.mean(obj_values), np.mean(known_obj), np.mean(opt_gap), np.mean(ineq_max_vals), np.mean(ineq_mean_vals), np.mean([0.0]), np.mean([0.0])

def dual_evaluate(data, dual_net, test_indices):
    X = data.X[test_indices]
    Y_target = data.opt_targets["y_operational"][test_indices]
    # target_mu = data.mu[test_indices]
    # target_lamb = data.lamb[test_indices]
    target_mu = data.opt_targets["mu_operational"][test_indices]  
    target_lamb = data.opt_targets["lamb_operational"][test_indices]


    # Forward pass through networks
    dual_net.eval()
    mu, lamb = dual_net(X)

    obj_values = data.dual_obj_fn(X, mu, lamb).detach().numpy()
    known_obj = data.dual_obj_fn(X, target_mu, target_lamb).detach().numpy()
    # known_obj = data.obj_fn(X, Y_target).detach().numpy()

    # print(np.allclose(known_dual_obj, known_obj, 1e-10))

    # dual_ineq_dist = data.dual_ineq_dist(mu, lamb)
    dual_ineq_resid = data.dual_ineq_resid(mu, lamb)
    dual_ineq_dist = torch.clamp(dual_ineq_resid, 0)
    dual_eq_resid = data.dual_eq_resid(mu, lamb)

    opt_gap = (obj_values - known_obj)/np.abs(known_obj) * 100

    ineq_max_vals = torch.max(dual_ineq_dist, dim=1)[0].detach().numpy() # First element is the max, second is the index
    eq_max_vals = torch.max(torch.abs(dual_eq_resid), dim=1)[0].detach().numpy() # First element is the max, second is the index
    ineq_mean_vals = torch.mean(dual_ineq_dist, dim=1).detach().numpy()
    eq_mean_vals = torch.mean(torch.abs(dual_eq_resid), dim=1).detach().numpy()

    return np.mean(obj_values), np.mean(known_obj), np.mean(opt_gap), np.mean(ineq_max_vals), np.mean(ineq_mean_vals), np.mean(eq_max_vals), np.mean(eq_mean_vals)

repeats = 5
stats_dict = {}

# Get the quality of dual solutions of QP:
dual_stats_dict = {}
for experiment in ["simple"]:
    dual_stats_dict[experiment] = {"obj": [], "known_obj": [], "opt_gap": [], "ineq_max": [], "ineq_mean": [], "eq_max": [], "eq_mean": []}
    for repeat in range(repeats):
        directory = f"experiment-output/ch4/ch4-reproduction/{experiment}/repeat:{repeat}"
        data_path = f"data/QP_data/QP_type:{experiment}_var:100_ineq:50_eq:50_num_samples:10000.pkl"

        qp_args = json.load(open('config.json'))
        data = pickle.load(open(data_path, 'rb'))
        dual_net = DualNet(qp_args, data=data)
        dual_net.load_state_dict(torch.load(os.path.join(directory, 'dual_weights.pth'), weights_only=True))

        obj_val, known_obj, opt_gap, ineq_max, ineq_mean, eq_max, eq_mean = dual_evaluate_QP(data, dual_net)
        dual_stats_dict[experiment]["obj"].append(obj_val)
        dual_stats_dict[experiment]["known_obj"].append(known_obj)
        dual_stats_dict[experiment]["opt_gap"].append(opt_gap)
        dual_stats_dict[experiment]["ineq_max"].append(ineq_max)
        dual_stats_dict[experiment]["ineq_mean"].append(ineq_mean)
        dual_stats_dict[experiment]["eq_max"].append(eq_max)
        dual_stats_dict[experiment]["eq_mean"].append(eq_mean)


data_path = "experiment-output/ch7/ED_NB-G-F_GB2-G2-F2_L3_c0_s0_p0_smp15.pkl"
data = pickle.load(open(data_path, 'rb'))

indices = torch.arange(data.X.shape[0])

# ---- 7.1 -----    
# Get the quality of dual solutions of ED:
for run, (path, name) in enumerate(zip(
    ["experiment-output/ch5/plain-PDL/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746265816-296861", 
    "experiment-output/ch5/repair-1/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746429988-424426",
    "experiment-output/ch5/repair-2/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746434902-974866"],
    ["plain", "repair1", "repair2"])):
    dual_stats_dict[name] = {"obj": [], "known_obj": [], "opt_gap": [], "ineq_max": [], "ineq_mean": [], "eq_max": [], "eq_mean": []}
    for repeat in range(repeats):
        directory = os.path.join(path, f"repeat:{repeat}")
        with open(os.path.join(path, 'args.json'), 'r') as f:
            args = json.load(f)
        # Compute sizes for each set
        train_size = int(args["train"] * data.X.shape[0])
        valid_size = int(args["valid"] * data.X.shape[0])

        

        train_indices = indices[:train_size]
        valid_indices = indices[train_size:train_size+valid_size]
        test_indices = indices[train_size+valid_size:]


        data = pickle.load(open(data_path, 'rb'))
        dual_net = DualNet(args, data=data)
        dual_net.load_state_dict(torch.load(os.path.join(directory, 'dual_weights.pth'), weights_only=True))

        obj_val, known_obj, opt_gap, ineq_max, ineq_mean, eq_max, eq_mean = dual_evaluate(data, dual_net, test_indices)
        dual_stats_dict[name]["obj"].append(obj_val)
        dual_stats_dict[name]["known_obj"].append(known_obj)
        dual_stats_dict[name]["opt_gap"].append(opt_gap)
        dual_stats_dict[name]["ineq_max"].append(ineq_max)
        dual_stats_dict[name]["ineq_mean"].append(ineq_mean)
        dual_stats_dict[name]["eq_max"].append(eq_max)
        dual_stats_dict[name]["eq_mean"].append(eq_mean)

dual_stats_dict

{'simple': {'obj': [-1328.131182337341,
   -1646.084331887411,
   -14147.795381650087,
   -5609.650842847485,
   -16158.08999932075],
  'known_obj': [-15.037240915640544,
   -15.037240915640544,
   -15.037240915640544,
   -15.037240915640544,
   -15.037240915640544],
  'opt_gap': [-8744.36099506512,
   -10863.43555703428,
   -94153.19499255384,
   -37259.73654047795,
   -107712.75419794535],
  'ineq_max': [0.01680701524556039,
   0.020797784714214118,
   0.03499553406049418,
   0.02380447189877212,
   0.018363942571592826],
  'ineq_mean': [0.0016263657114894976,
   0.0023613674324203214,
   0.0025684040018297155,
   0.0034214609907723837,
   0.0027961065415661674],
  'eq_max': [0.0, 0.0, 0.0, 0.0, 0.0],
  'eq_mean': [0.0, 0.0, 0.0, 0.0, 0.0]},
 'plain': {'obj': [830078572046.8887,
   840901026473.9683,
   839805885523.9711,
   820386304497.4811,
   824215545349.2648],
  'known_obj': [29293.887262458353,
   29293.887262458353,
   29293.887262458353,
   29293.887262458353,
   29293.88726

In [26]:
import numpy as np
import pandas as pd

def format_sci_mean_std(mean, std):
    """Format mean and std as 'mantissa (std) e exponent' with 1 decimal, mantissa always in [1.0, 10.0)."""
    def get_mantissa_exp(val):
        if val == 0:
            return 0.0, 0
        exponent = int(np.floor(np.log10(abs(val))))
        mantissa = val / (10 ** exponent)
        return mantissa, exponent

    # Use mean or std for exponent reference
    ref_val = mean if mean != 0 else std
    mantissa_mean, exponent = get_mantissa_exp(ref_val)
    mantissa_std = std / (10 ** exponent) if ref_val != 0 else 0.0
    return f"{mantissa_mean:.1f} ({mantissa_std:.1f}) e{exponent}"

def format_orig_mean_std(mean, std):
    """Format mean and std as 'mean (std)' with 3 decimals."""
    return f"{mean:.3f} ({std:.3f})"

# Prepare final summary for LaTeX export
summary = {
    "Predicted Obj": [],
    "OptGap (\%)": [],
    "IneqMax": [],
    "IneqMean": [],
    "EqMax": [],
    "EqMean": [],
}
summary_orig = {
    "Predicted Obj": [],
    "OptGap (\%)": [],
    "IneqMax": [],
    "IneqMean": [],
    "EqMax": [],
    "EqMean": [],
}

for experiment, metrics in dual_stats_dict.items():
    # Predicted Obj (scientific)
    mean = np.mean(metrics['obj'])
    std = np.std(metrics['obj'])
    summary["Predicted Obj"].append(format_sci_mean_std(mean, std))
    summary_orig["Predicted Obj"].append(format_orig_mean_std(mean, std))
    # OptGap (scientific)
    mean = np.mean(metrics['opt_gap'])
    std = np.std(metrics['opt_gap'])
    summary["OptGap (\%)"].append(format_sci_mean_std(mean, std))
    summary_orig["OptGap (\%)"].append(format_orig_mean_std(mean, std))
    # IneqMax (scientific)
    mean = np.mean(metrics['ineq_max'])
    std = np.std(metrics['ineq_max'])
    summary["IneqMax"].append(format_sci_mean_std(mean, std))
    summary_orig["IneqMax"].append(format_orig_mean_std(mean, std))
    # IneqMean (scientific)
    mean = np.mean(metrics['ineq_mean'])
    std = np.std(metrics['ineq_mean'])
    summary["IneqMean"].append(format_sci_mean_std(mean, std))
    summary_orig["IneqMean"].append(format_orig_mean_std(mean, std))
    # EqMax (scientific)
    mean = np.mean(metrics['eq_max'])
    std = np.std(metrics['eq_max'])
    summary["EqMax"].append(format_sci_mean_std(mean, std))
    summary_orig["EqMax"].append(format_orig_mean_std(mean, std))
    # EqMean (scientific)
    mean = np.mean(metrics['eq_mean'])
    std = np.std(metrics['eq_mean'])
    summary["EqMean"].append(format_sci_mean_std(mean, std))
    summary_orig["EqMean"].append(format_orig_mean_std(mean, std))

# Formatted table
df = pd.DataFrame(summary)
df_T = df.T
df_T.columns = list(dual_stats_dict.keys())  # Set experiment names as columns
df_T = df_T.reset_index().rename(columns={'index': 'Statistic'})

# Unformatted mean/std table
df_orig = pd.DataFrame(summary_orig)
df_orig_T = df_orig.T
df_orig_T.columns = list(dual_stats_dict.keys())
df_orig_T = df_orig_T.reset_index().rename(columns={'index': 'Statistic'})

# Print LaTeX tables
print("Formatted (scientific) table:")
print(df_T.to_latex(index=False, escape=False))

print("\nOriginal mean (std) table:")
print(df_orig_T.to_latex(index=False, escape=False))

Formatted (scientific) table:
\begin{tabular}{lllll}
\toprule
Statistic & simple & plain & repair1 & repair2 \\
\midrule
Predicted Obj & -7.8 (6.2) e3 & 8.3 (0.8) e11 & 3.4 (6.6) e0 & -1.0 (2.6) e0 \\
OptGap (\%) & -5.2 (4.2) e4 & 3.0 (0.2) e10 & -10.0 (0.0) e1 & -1.0 (0.0) e2 \\
IneqMax & 2.3 (6.5) e-2 & 2.4 (2.4) e5 & 5.1 (1.0) e-4 & 4.2 (8.3) e-4 \\
IneqMean & 2.6 (5.8) e-3 & 1.6 (1.5) e4 & 5.4 (1.1) e-5 & 1.7 (3.5) e-5 \\
EqMax & 0.0 (0.0) e0 & 1.1 (0.2) e7 & 1.0 (0.0) e1 & 1.0 (0.0) e1 \\
EqMean & 0.0 (0.0) e0 & 6.4 (1.0) e6 & 2.5 (0.0) e0 & 2.5 (0.0) e0 \\
\bottomrule
\end{tabular}


Original mean (std) table:
\begin{tabular}{lllll}
\toprule
Statistic & simple & plain & repair1 & repair2 \\
\midrule
Predicted Obj & -7777.950 (6240.243) & 831077466778.315 (8186236329.250) & 3.385 (6.559) & -1.002 (2.579) \\
OptGap (\%) & -51746.696 (41619.766) & 29589132754.143 (171565067.315) & -99.995 (0.010) & -100.128 (0.274) \\
IneqMax & 0.023 (0.006) & 235273.921 (24166.547) & 0.001 (0.001) 

In [22]:
# ---- 7.5 -----    
# Get the quality of dual solutions:
from networks import DualClassificationNetEndToEnd


dual_stats_dict = {}
for run, (path, name) in enumerate(zip(
    [
        "experiment-output/ch6/3-nodes/dual-completion/newest_run", 
        "experiment-output/ch6/3-nodes/dual-classification/newest_run"
    ],
    [
        "completion",
        "classification"
    ])):
    dual_stats_dict[name] = {"obj": [], "known_obj": [], "opt_gap": [], "ineq_max": [], "ineq_mean": [], "eq_max": [], "eq_mean": []}
    for repeat in range(repeats):
        directory = os.path.join(path, f"repeat:{repeat}")
        with open(os.path.join(path, 'args.json'), 'r') as f:
            args = json.load(f)
        # Compute sizes for each set
        train_size = int(args["train"] * data.X.shape[0])
        valid_size = int(args["valid"] * data.X.shape[0])

        train_indices = indices[:train_size]
        valid_indices = indices[train_size:train_size+valid_size]
        test_indices = indices[train_size+valid_size:]


        data = pickle.load(open(data_path, 'rb'))
        if run == 0:
            dual_net = DualNetEndToEnd(args, data=data)
        else:
            dual_net = DualClassificationNetEndToEnd(args, data=data)
        dual_net.load_state_dict(torch.load(os.path.join(directory, 'dual_weights.pth'), weights_only=True))

        obj_val, known_obj, opt_gap, ineq_max, ineq_mean, eq_max, eq_mean = dual_evaluate(data, dual_net, test_indices)
        dual_stats_dict[name]["obj"].append(obj_val)
        dual_stats_dict[name]["known_obj"].append(known_obj)
        dual_stats_dict[name]["opt_gap"].append(opt_gap)
        dual_stats_dict[name]["ineq_max"].append(ineq_max)
        dual_stats_dict[name]["ineq_mean"].append(ineq_mean)
        dual_stats_dict[name]["eq_max"].append(eq_max)
        dual_stats_dict[name]["eq_mean"].append(eq_mean)

dual_stats_dict

{'completion': {'obj': [28415.2342771398,
   28276.766905426706,
   28453.742061615216,
   28284.535654657113,
   28438.550995380392],
  'known_obj': [29293.887262458353,
   29293.887262458353,
   29293.887262458353,
   29293.887262458353,
   29293.887262458353],
  'opt_gap': [-56.21245468110735,
   -59.473604765918445,
   -52.62728674649323,
   -62.60254343063131,
   -50.55739209099471],
  'ineq_max': [0.0, 0.0, 0.0, 0.0, 0.0],
  'ineq_mean': [0.0, 0.0, 0.0, 0.0, 0.0],
  'eq_max': [9.795400321540753e-17,
   9.703148459371286e-17,
   9.667414521768291e-17,
   9.726586724075517e-17,
   9.681970465135701e-17],
  'eq_mean': [1.141142976140631e-17,
   1.131934072525713e-17,
   1.126416558133277e-17,
   1.1350389642146944e-17,
   1.1279303302139569e-17]},
 'classification': {'obj': [29202.436281227412,
   29043.63723645002,
   29177.512451691306,
   25260.95882888781,
   27609.171883932922],
  'known_obj': [29293.887262458353,
   29293.887262458353,
   29293.887262458353,
   29293.887262458

In [23]:
# Prepare final summary for LaTeX export
summary = {
    # "Optimal Obj": [],
    "Predicted Obj": [],
    "OptGap (\%)": [],
    "IneqMax": [],
    "IneqMean": [],
    "EqMax": [],
    "EqMean": [],
}

for experiment, metrics in dual_stats_dict.items():
    # summary["Optimal Obj"].append(f"{np.mean(metrics['known_obj']):.3e}")
    summary["Predicted Obj"].append(f"{np.mean(metrics['obj']):.3f}({np.std(metrics['obj']):.3f})")
    summary["OptGap (\%)"].append(f"{np.mean(metrics['opt_gap']):.3f}({np.std(metrics['opt_gap']):.3f})")
    summary["IneqMax"].append(
        f"{np.mean(metrics['ineq_max']):.3f}({np.std(metrics['ineq_max']):.3f})"
    )
    summary["IneqMean"].append(
        f"{np.mean(metrics['ineq_mean']):.3f}({np.std(metrics['ineq_mean']):.3f})"
    )
    summary["EqMax"].append(
        f"{np.mean(metrics['eq_max']):.3f}({np.std(metrics['eq_max']):.3f})"
    )
    summary["EqMean"].append(
        f"{np.mean(metrics['eq_mean']):.3f}({np.std(metrics['eq_mean']):.3f})"
    )

df = pd.DataFrame(summary)

# For LaTeX export
latex_table = df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{llllll}
\toprule
Predicted Obj & OptGap (\%) & IneqMax & IneqMean & EqMax & EqMean \\
\midrule
28373.766(77.050) & -56.295(4.388) & 0.000(0.000) & 0.000(0.000) & 0.000(0.000) & 0.000(0.000) \\
28058.743(1520.487) & -20.722(29.780) & 0.000(0.000) & 0.000(0.000) & 0.000(0.000) & 0.000(0.000) \\
\bottomrule
\end{tabular}



In [7]:
# Select the experiments you want
experiments = ['completion', 'classification']  # or any two you want

# Build the table data
opt_gap_data = []
for exp in experiments:
    # Get the list of opt_gap values for this experiment
    gaps = dual_stats_dict[exp]['opt_gap']
    # Format as strings if you want, or just keep as floats
    opt_gap_data.append(gaps)

# Create DataFrame: rows=experiments, columns=run indices
df_optgap = pd.DataFrame(opt_gap_data, index=experiments, columns=[f'Run {i+1}' for i in range(5)])

# Reset index to have 'Experiment' as a column
df_optgap = df_optgap.reset_index().rename(columns={'index': 'Experiment'})

# For LaTeX export
latex_table = df_optgap.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{lrrrrr}
\toprule
Experiment & Run 1 & Run 2 & Run 3 & Run 4 & Run 5 \\
\midrule
completion & -56.212455 & -59.473605 & -52.627287 & -62.602543 & -50.557392 \\
classification & -3.895719 & -10.456890 & -3.858370 & -5.313673 & -80.085375 \\
\bottomrule
\end{tabular}

